In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import usaddress
import time

C:\Users\Public\Anaconda\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
#Load Raw Data
file_name = r'Evictions_Raw.csv'
data = pd.read_csv(file_name)

In [4]:
data.head()

COURT_INDEX_NUMBER  DOCKET_NUMBER             EVICTION_ADDRESS  \
0           50180/17           5469  463 LEXINGTON AVENUE APT. 1   
1          73750/18B          93847          448 SAPPHIRE STREET   
2          K65227/17          97709            1036 PRESIDENT ST   
3           51881/18         350940         2124 ST. PAUL AVENUE   
4           66981/18         351548            1098 GRANT AVENUE   

  EVICTION_APT_NUM EXECUTED_DATE MARSHAL_FIRST_NAME MARSHAL_LAST_NAME  \
0                1    09/12/2017             Robert          Renzulli   
1                1    06/03/2019             Justin          Grossman   
2              B2B    11/16/2017            Darlene            Barone   
3        2ND FLOOR    02/05/2019             Thomas               Bia   
4                2    06/07/2019             Thomas               Bia   

  RESIDENTIAL_COMMERCIAL_IND   BOROUGH  EVICTION_ZIP  
0                Residential  BROOKLYN         11221  
1                Residential  BROOKLYN         11208  
2                Residential  BROOKLYN         11225  
3                Residential     BRONX         10461  
4                Residential     BRONX         10456

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66349 entries, 0 to 66348
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   COURT_INDEX_NUMBER          66349 non-null  object
 1   DOCKET_NUMBER               66349 non-null  int64 
 2   EVICTION_ADDRESS            66349 non-null  object
 3   EVICTION_APT_NUM            55378 non-null  object
 4   EXECUTED_DATE               66349 non-null  object
 5   MARSHAL_FIRST_NAME          66349 non-null  object
 6   MARSHAL_LAST_NAME           66349 non-null  object
 7   RESIDENTIAL_COMMERCIAL_IND  66349 non-null  object
 8   BOROUGH                     66349 non-null  object
 9   EVICTION_ZIP                66349 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 5.1+ MB


In [5]:
data.isna().mean()

COURT_INDEX_NUMBER            0.000000
DOCKET_NUMBER                 0.000000
EVICTION_ADDRESS              0.000000
EVICTION_APT_NUM              0.165353
EXECUTED_DATE                 0.000000
MARSHAL_FIRST_NAME            0.000000
MARSHAL_LAST_NAME             0.000000
RESIDENTIAL_COMMERCIAL_IND    0.000000
BOROUGH                       0.000000
EVICTION_ZIP                  0.000000
dtype: float64

In [6]:
#use address module to format addresses, keep important information, help identify errors
def addressparse(raw_address):
    try:
        new_address = usaddress.tag(raw_address)[0]
    except:
        new_address = np.nan
    return new_address

In [7]:
data['parsed_address']=data['EVICTION_ADDRESS'].apply(lambda x:addressparse(x))

In [8]:
#example result
data['parsed_address'][0]

OrderedDict([('AddressNumber', '463'),
             ('StreetName', 'LEXINGTON'),
             ('StreetNamePostType', 'AVENUE'),
             ('OccupancyType', 'APT.'),
             ('OccupancyIdentifier', '1')])

In [9]:
#List different address tags generated by address module
parsed_address_parts=data['parsed_address'].apply(pd.Series).columns

In [10]:
list(parsed_address_parts)

[0,
 'AddressNumber',
 'AddressNumberPrefix',
 'AddressNumberSuffix',
 'BuildingName',
 'IntersectionSeparator',
 'LandmarkName',
 'NotAddress',
 'OccupancyIdentifier',
 'OccupancyType',
 'PlaceName',
 'Recipient',
 'SecondStreetName',
 'SecondStreetNamePostDirectional',
 'SecondStreetNamePostType',
 'StateName',
 'StreetName',
 'StreetNamePostDirectional',
 'StreetNamePostModifier',
 'StreetNamePostType',
 'StreetNamePreDirectional',
 'StreetNamePreType',
 'SubaddressIdentifier',
 'SubaddressType',
 'USPSBoxID',
 'USPSBoxType',
 'ZipCode']

In [11]:
#Select key address information
good_vars=['AddressNumber','StreetNamePreDirectional','StreetNamePreType','StreetName','StreetNamePostType','StreetNamePostDirectional']
def generate_clean_address(raw_dict):
    new_address = ''
    try:
        for var in good_vars:
            if var in list(raw_dict.keys()):
                if raw_dict[var] !=np.nan:
                    new_address=new_address+' '+str(raw_dict[var])
    except:
        new_address=np.nan
    return new_address

In [12]:
data['address_cleaned']=data['parsed_address'].apply(lambda x:generate_clean_address(x))

In [13]:
data['address_cleaned'][0:10]

0      463 LEXINGTON AVENUE
1       448 SAPPHIRE STREET
2         1036 PRESIDENT ST
3      2124 ST. PAUL AVENUE
4         1098 GRANT AVENUE
5       413 WESTERVELT AVE.
6        485 JACKSON AVENUE
7      239 EAST 80TH STREET
8     424 BRONX PARK AVENUE
9         1449 GATES AVENUE
Name: address_cleaned, dtype: object

In [14]:
data.isna().mean()

COURT_INDEX_NUMBER            0.000000
DOCKET_NUMBER                 0.000000
EVICTION_ADDRESS              0.000000
EVICTION_APT_NUM              0.165353
EXECUTED_DATE                 0.000000
MARSHAL_FIRST_NAME            0.000000
MARSHAL_LAST_NAME             0.000000
RESIDENTIAL_COMMERCIAL_IND    0.000000
BOROUGH                       0.000000
EVICTION_ZIP                  0.000000
parsed_address                0.035358
address_cleaned               0.035358
dtype: float64

In [15]:
data[data['parsed_address'].isna()]

COURT_INDEX_NUMBER  DOCKET_NUMBER  \
208            68376/18A         487928   
223             68673/18          22706   
239             53168/18           9901   
287             33027/19         100017   
306             61218/18          81959   
...                  ...            ...   
66246          900506/18          83389   
66276           55548/18          79806   
66285           66469/19          25488   
66320           72327/16         474140   
66327           91707/17           7112   

                                        EVICTION_ADDRESS    EVICTION_APT_NUM  \
208              420 EAST 61ST ST. AK A ONE SUTTON PL NO                  3C   
223       735 EAST 166TH STREET A/K/A 1095 FOREST AVENUE                  5D   
239        429 RIDGEWOOD AVE - 1ST FLOOR APT IN THE BACK                 NaN   
287    2006 ELLIS AVENUE A/ K/A 2006 CROSS BRONX EXPR...                   D   
306            94-07 83RD STREET A/ K/A 94-07 82ND PLACE              UNIT 2   
...                                                  ...                 ...   
66246  2387 MORRIS AVE. A/K/A 67-83 EAST 184TH STREET...  STORE #1 PLUS BSMT   
66276  869 THOMAS S. BOYLAN D STREET A/K/A 869 HOPKIN...                  3C   
66285  95-13/15/17 NORTHERN BLVD A/K/A 32-56 96TH STREET                 NaN   
66320           39 SICKLES STREET AK A 29-45 SICKLES ST.                  2J   
66327  474-76 48TH STREET A/K/A 4802 5TH AVENUE - 4TH FL                 NaN   

      EXECUTED_DATE MARSHAL_FIRST_NAME MARSHAL_LAST_NAME  \
208      04/29/2019              Danny          Weinheim   
223      05/09/2019             Edward             Guida   
239      01/04/2019             Robert          Renzulli   
287      10/24/2019             Justin          Grossman   
306      07/20/2018             Justin          Grossman   
...             ...                ...               ...   
66246    06/21/2018              Henry             Daley   
66276    06/01/2018             Justin          Grossman   
66285    03/10/2020             Edward             Guida   
66320    10/26/2017              Danny          Weinheim   
66327    01/02/2018             Robert          Renzulli   

      RESIDENTIAL_COMMERCIAL_IND        BOROUGH  EVICTION_ZIP parsed_address  \
208                  Residential      MANHATTAN         10065            NaN   
223                  Residential          BRONX         10456            NaN   
239                  Residential  STATEN ISLAND         10312            NaN   
287                  Residential          BRONX         10472            NaN   
306                  Residential         QUEENS         11416            NaN   
...                          ...            ...           ...            ...   
66246                 Commercial          BRONX         10468            NaN   
66276                Residential       BROOKLYN         11212            NaN   
66285                 Commercial         QUEENS         11372            NaN   
66320                Residential      MANHATTAN         10040            NaN   
66327                 Commercial       BROOKLYN         11220            NaN   

      address_cleaned  
208               NaN  
223               NaN  
239               NaN  
287               NaN  
306               NaN  
...               ...  
66246             NaN  
66276             NaN  
66285             NaN  
66320             NaN  
66327             NaN  

[2346 rows x 12 columns]

In [21]:
#certain words cause problems for the address module or the module is fed two addresses with aka.
#split and select first section
splitter_list=['AKA', 'A/K/A', 'A/, K/A', 'A /K/A','APT', 'A/ K/A','A/K /A','A/K/ A','A KA','AK A','B/K/A','UNIT','BUILDING', 'FLOOR','ROOM',
              'ENTRANCE','BULD','FL','ENTIRE','AS PER','STORE','GARAGE','\(','PARKING SPACE','PORTION OF','IN THE BLDG','LOT#1','LOEW LVL','AK 3594',
               '120 NEWEL STR','BSMT','RIGHT SIDE','LEFT SIDE','LOT 6','- 2ND AND','- PORTIONS','A/KA','A.K.A.', ',CO RNER',': ','LOT E - SPACE','/5',
               'N. GOETHALS ','38-34 13TH ST. AS PER','LOT L - SPACE','BOOTHS#','IN A PREMISES','2ND AND 3RD','LOT 1, SPOT 2','FRONT DOOR','APPROX','LOT ',
               'IN THE BLDG','PARKING ','1ST AND 2ND','THE BLDG','SUITE ','IN THE BLD','IN BKA','KNOWN ','ON ','3RD AND 4TH','BSMNT','BKA ','LEFT FRONT',
               'BUILDI','AND ','ALL ','AN D','IN TH ','BDLG','.SO.','IN THE BLDG','NORTHEAST CORNER']
data['eviction_address2']=np.nan
for splitter in splitter_list:
    data['eviction_address2']=np.where(((data['parsed_address'].isna() & (data['address_cleaned'].isna()) & (data['EVICTION_ADDRESS'].str.contains(splitter)))),
                                       data['EVICTION_ADDRESS'].str.split(splitter).str.slice(stop=1).str.join(sep=''),data['eviction_address2'])
    
    data['eviction_address2']=data['eviction_address2'].replace('nan',np.nan)
    
    data['parsed_address']=np.where((data['parsed_address'].isna() & (~data['eviction_address2'].isna())),
                                data['eviction_address2'].apply(lambda x:addressparse(x)),data['parsed_address'])
    
    data['address_cleaned']=np.where(data['address_cleaned'].isna() & (~data['parsed_address'].isna()),
                                     data['parsed_address'].apply(lambda x:generate_clean_address(x)), data['address_cleaned'])
    data['eviction_address2']=np.where(data['address_cleaned'].isna(),np.nan,data['eviction_address2'])

In [22]:
#Remaining Errors
data[data['parsed_address'].isna()]

COURT_INDEX_NUMBER  DOCKET_NUMBER  \
6625          B900783/17         381787   
20459          901264/17          72694   
21664           54821/16            803   
28001           67734/18          33804   
41996           72385/18         485910   
44470          K63535/17          79937   
47416         13182/15-1         204117   
52188          901800/17          76224   
57672         13182/15-2         198169   
58531           57909/19          23523   
59693           57521/19          17473   

                                        EVICTION_ADDRESS  \
6625                 333 EAST 181ST ST 355 EAST 184TH ST   
20459                    1972&1974&1976 PROSP ECT AVENUE   
21664  218-26,28A,28B MERRI CK BLVD  IN THE BLDG KNOW...   
28001              200-04 46TH ROAD ENT. 1 FAM. DWELLING   
41996           4927 BROADWAY IN THE  BLDG KA 616 WEST *   
44470                   1000 JAMAICA AVE 998 JAMAICA AVE   
47416  126-74,126-76,126-82 WILLETS POINT BLVD BUILDI...   
52188  531 COSTER ST IN  TH E BLDG. KNOWN AS 531-537 ...   
57672  126-74 126-76 126-82 WILLETS POINT BLVD BUILDI...   
58531                  91-01 /91-03/ 91-05 ASTORIA BLVD.   
59693                   912 SEVENTH AVE 200 WEST 58TH ST   

           EVICTION_APT_NUM EXECUTED_DATE MARSHAL_FIRST_NAME  \
6625                   COMM    08/22/2018            Richard   
20459  UNIT:%SUPERMARKET/GR    11/15/2018             Justin   
21664                   NaN    07/30/2018         Salavatore   
28001                   NaN    12/04/2018              Bruce   
41996                     *    02/01/2019              Danny   
44470            STOREFRONT    08/29/2017             Ileana   
47416                   NaN    09/17/2018            Richard   
52188             1ST FLOOR    09/11/2018             Justin   
57672                   NaN    09/17/2018            Richard   
58531                   NaN    06/05/2019             Edward   
59693                   NaN    10/07/2019             George   

      MARSHAL_LAST_NAME RESIDENTIAL_COMMERCIAL_IND    BOROUGH  EVICTION_ZIP  \
6625              McCoy                Residential      BRONX         10458   
20459          Grossman                 Commercial      BRONX         10460   
21664            Giglio                 Commercial     QUEENS         11413   
28001              Kemp                Residential     QUEENS         11361   
41996          Weinheim                 Commercial  MANHATTAN         10034   
44470            Rivera                 Commercial   BROOKLYN         11208   
47416           Capuano                 Commercial     QUEENS         11368   
52188          Grossman                 Commercial      BRONX         10474   
57672           Capuano                 Commercial     QUEENS         11368   
58531             Guida                 Commercial     QUEENS         11369   
59693       Essock, Jr.                 Commercial  MANHATTAN         10019   

      parsed_address address_cleaned eviction_address2  
6625             NaN             NaN               NaN  
20459            NaN             NaN               NaN  
21664            NaN             NaN               NaN  
28001            NaN             NaN               NaN  
41996            NaN             NaN               NaN  
44470            NaN             NaN               NaN  
47416            NaN             NaN               NaN  
52188            NaN             NaN               NaN  
57672            NaN             NaN               NaN  
58531            NaN             NaN               NaN  
59693            NaN             NaN               NaN

In [23]:
data['parsed_address'].isna().mean()

0.00016578998929900977

In [24]:
data['address_cleaned'].isna().mean()

0.00016578998929900977

In [26]:
data['address_cleaned']

0         463 LEXINGTON AVENUE
1          448 SAPPHIRE STREET
2            1036 PRESIDENT ST
3         2124 ST. PAUL AVENUE
4            1098 GRANT AVENUE
                 ...          
66344                         
66345     150 HENDRICKS AVENUE
66346      237 OVINGTON AVENUE
66347      790 RIVERSIDE DRIVE
66348     358 KOSCIUSKO STREET
Name: address_cleaned, Length: 66349, dtype: object

In [27]:
data['full_address']= data['address_cleaned'] + ', ' + data['BOROUGH'] + ', NY, ' +data['EVICTION_ZIP'].astype(str)

In [28]:
data['full_address']

0              463 LEXINGTON AVENUE, BROOKLYN, NY, 11221
1               448 SAPPHIRE STREET, BROOKLYN, NY, 11208
2                 1036 PRESIDENT ST, BROOKLYN, NY, 11225
3                 2124 ST. PAUL AVENUE, BRONX, NY, 10461
4                    1098 GRANT AVENUE, BRONX, NY, 10456
                              ...                       
66344                                , QUEENS, NY, 11368
66345     150 HENDRICKS AVENUE, STATEN ISLAND, NY, 10301
66346           237 OVINGTON AVENUE, BROOKLYN, NY, 11209
66347          790 RIVERSIDE DRIVE, MANHATTAN, NY, 10032
66348          358 KOSCIUSKO STREET, BROOKLYN, NY, 11211
Name: full_address, Length: 66349, dtype: object

In [29]:
data['full_address'].isna().mean()

0.00016578998929900977

In [30]:
data.to_csv('cleaned_addresses_evictions.csv')

with addresses cleaned, the next step is to get coordinates to match to census tract boundaries

In [32]:
file_location = r'C:\Users\Adam Parente\Documents\Python Projects\new evictions\cleaned_addresses_evictions.csv'
data = pd.read_csv(file_location,index_col='Unnamed: 0')

C:\Users\Public\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
data.head()

COURT_INDEX_NUMBER  DOCKET_NUMBER             EVICTION_ADDRESS  \
0           50180/17           5469  463 LEXINGTON AVENUE APT. 1   
1          73750/18B          93847          448 SAPPHIRE STREET   
2          K65227/17          97709            1036 PRESIDENT ST   
3           51881/18         350940         2124 ST. PAUL AVENUE   
4           66981/18         351548            1098 GRANT AVENUE   

  EVICTION_APT_NUM EXECUTED_DATE MARSHAL_FIRST_NAME MARSHAL_LAST_NAME  \
0                1    09/12/2017             Robert          Renzulli   
1                1    06/03/2019             Justin          Grossman   
2              B2B    11/16/2017            Darlene            Barone   
3        2ND FLOOR    02/05/2019             Thomas               Bia   
4                2    06/07/2019             Thomas               Bia   

  RESIDENTIAL_COMMERCIAL_IND   BOROUGH  EVICTION_ZIP  \
0                Residential  BROOKLYN         11221   
1                Residential  BROOKLYN         11208   
2                Residential  BROOKLYN         11225   
3                Residential     BRONX         10461   
4                Residential     BRONX         10456   

                                      parsed_address        address_cleaned  \
0  OrderedDict([('AddressNumber', '463'), ('Stree...   463 LEXINGTON AVENUE   
1  OrderedDict([('AddressNumber', '448'), ('Stree...    448 SAPPHIRE STREET   
2  OrderedDict([('AddressNumber', '1036'), ('Stre...      1036 PRESIDENT ST   
3  OrderedDict([('AddressNumber', '2124'), ('Stre...   2124 ST. PAUL AVENUE   
4  OrderedDict([('AddressNumber', '1098'), ('Stre...      1098 GRANT AVENUE   

  eviction_address2                                full_address  
0               NaN   463 LEXINGTON AVENUE, BROOKLYN, NY, 11221  
1               NaN    448 SAPPHIRE STREET, BROOKLYN, NY, 11208  
2               NaN      1036 PRESIDENT ST, BROOKLYN, NY, 11225  
3               NaN      2124 ST. PAUL AVENUE, BRONX, NY, 10461  
4               NaN         1098 GRANT AVENUE, BRONX, NY, 10456

In [34]:
#geocoder function
def get_point(address):
    try:
        raw = geocoder.arcgis(address)
        if raw.status !='OK':
            return np.nan
        else:
            location = (raw.lat,raw.lng)
            return location
    except:
        return np.nan

In [35]:
#geocoder test
get_point(data['full_address'][0])

(40.68865999611498, -73.94358600207771)

In [36]:
#warning, this will take hours to complete.  if you have access to better geocoders behind paywall, use those to speed it up.
data['lat_lon']=data['full_address'].apply(lambda x: get_point(x))

Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Max retries exceeded with url: /arcgis/rest/services/World/GeocodeServer/find?f=json&text=+2105+RYER+AVENUE%2C+BRONX%2C+NY%2C+10457&maxLocations=1 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x0000011B39F90588>, 'Connection to geocode.arcgis.com timed out. (connect timeout=5.0)'))
Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPS

In [142]:
data.to_csv('evictions_geocoded_1.csv')

In [146]:
file_location = r'C:\Users\Adam Parente\Documents\Python Projects\new evictions\evictions_geocoded_1.csv'
data = pd.read_csv(file_location,index_col='Unnamed: 0')

C:\Users\Public\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [148]:
data.head()

COURT_INDEX_NUMBER  DOCKET_NUMBER             EVICTION_ADDRESS  \
0           50180/17           5469  463 LEXINGTON AVENUE APT. 1   
1          73750/18B          93847          448 SAPPHIRE STREET   
2          K65227/17          97709            1036 PRESIDENT ST   
3           51881/18         350940         2124 ST. PAUL AVENUE   
4           66981/18         351548            1098 GRANT AVENUE   

  EVICTION_APT_NUM EXECUTED_DATE MARSHAL_FIRST_NAME MARSHAL_LAST_NAME  \
0                1    09/12/2017             Robert          Renzulli   
1                1    06/03/2019             Justin          Grossman   
2              B2B    11/16/2017            Darlene            Barone   
3        2ND FLOOR    02/05/2019             Thomas               Bia   
4                2    06/07/2019             Thomas               Bia   

  RESIDENTIAL_COMMERCIAL_IND   BOROUGH  EVICTION_ZIP  \
0                Residential  BROOKLYN         11221   
1                Residential  BROOKLYN         11208   
2                Residential  BROOKLYN         11225   
3                Residential     BRONX         10461   
4                Residential     BRONX         10456   

                                      parsed_address        address_cleaned  \
0  OrderedDict([('AddressNumber', '463'), ('Stree...   463 LEXINGTON AVENUE   
1  OrderedDict([('AddressNumber', '448'), ('Stree...    448 SAPPHIRE STREET   
2  OrderedDict([('AddressNumber', '1036'), ('Stre...      1036 PRESIDENT ST   
3  OrderedDict([('AddressNumber', '2124'), ('Stre...   2124 ST. PAUL AVENUE   
4  OrderedDict([('AddressNumber', '1098'), ('Stre...      1098 GRANT AVENUE   

  eviction_address2                                full_address  \
0               NaN   463 LEXINGTON AVENUE, BROOKLYN, NY, 11221   
1               NaN    448 SAPPHIRE STREET, BROOKLYN, NY, 11208   
2               NaN      1036 PRESIDENT ST, BROOKLYN, NY, 11225   
3               NaN      2124 ST. PAUL AVENUE, BRONX, NY, 10461   
4               NaN         1098 GRANT AVENUE, BRONX, NY, 10456   

                                    lat_lon  
0   (40.68865999611498, -73.94358600207771)  
1    (40.66745998368947, -73.8567509939227)  
2  (40.668459986647804, -73.95718496942762)  
3   (40.85483998565695, -73.82997902754228)  
4   (40.83090001290239, -73.91641002796423)

In [149]:
#Only a few errors
data['lat_lon'].isna().mean()

7.535908604500445e-05

In [150]:
#These should work, but a couple of errors isn't problematic.
data[data['lat_lon'].isna()]

COURT_INDEX_NUMBER  DOCKET_NUMBER      EVICTION_ADDRESS  \
32411           59490/16         462842      2105 RYER AVENUE   
45014           63770/17          63865   558 FOUNTAIN AVENUE   
46252          K64852/16          72210  368 EAST 26TH STREET   
58362          B23662/19         116727      186 CROSS STREET   
64929          B35564/18         109605    1495 BRYANT AVENUE   

      EVICTION_APT_NUM EXECUTED_DATE MARSHAL_FIRST_NAME MARSHAL_LAST_NAME  \
32411               2A    01/09/2017              Danny          Weinheim   
45014   2ND FLOOR REAR    07/05/2017             Justin          Grossman   
46252              NaN    02/15/2017             Ileana            Rivera   
58362               11    10/09/2019            Darlene            Barone   
64929    1ST FLOOR APT    11/28/2018            Darlene            Barone   

      RESIDENTIAL_COMMERCIAL_IND   BOROUGH  EVICTION_ZIP  \
32411                Residential     BRONX         10457   
45014                Residential  BROOKLYN         11207   
46252                Residential  BROOKLYN         11226   
58362                Residential     BRONX         10464   
64929                Residential     BRONX         10460   

                                          parsed_address  \
32411  OrderedDict([('AddressNumber', '2105'), ('Stre...   
45014  OrderedDict([('AddressNumber', '558'), ('Stree...   
46252  OrderedDict([('AddressNumber', '368'), ('Stree...   
58362  OrderedDict([('AddressNumber', '186'), ('Stree...   
64929  OrderedDict([('AddressNumber', '1495'), ('Stre...   

             address_cleaned eviction_address2  \
32411       2105 RYER AVENUE               NaN   
45014    558 FOUNTAIN AVENUE               NaN   
46252   368 EAST 26TH STREET               NaN   
58362       186 CROSS STREET               NaN   
64929     1495 BRYANT AVENUE               NaN   

                                     full_address lat_lon  
32411          2105 RYER AVENUE, BRONX, NY, 10457     NaN  
45014    558 FOUNTAIN AVENUE, BROOKLYN, NY, 11207     NaN  
46252   368 EAST 26TH STREET, BROOKLYN, NY, 11226     NaN  
58362          186 CROSS STREET, BRONX, NY, 10464     NaN  
64929        1495 BRYANT AVENUE, BRONX, NY, 10460     NaN

In [152]:
#try a different method to resolve errors
def get_point_nominatim(address):
    try:
        geo_nom = Nominatim(user_agent="eviction_test")
        raw = geo_nom.geocode(address)
        location = raw[1]
        return location
    except:
        return np.nan

In [153]:
sub_data = data[data['lat_lon'].isna()]

In [154]:
sub_data['lat_lon2']=sub_data['full_address'].apply(lambda x: get_point_nominatim(x))

C:\Users\Public\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [155]:
sub_data

COURT_INDEX_NUMBER  DOCKET_NUMBER      EVICTION_ADDRESS  \
32411           59490/16         462842      2105 RYER AVENUE   
45014           63770/17          63865   558 FOUNTAIN AVENUE   
46252          K64852/16          72210  368 EAST 26TH STREET   
58362          B23662/19         116727      186 CROSS STREET   
64929          B35564/18         109605    1495 BRYANT AVENUE   

      EVICTION_APT_NUM EXECUTED_DATE MARSHAL_FIRST_NAME MARSHAL_LAST_NAME  \
32411               2A    01/09/2017              Danny          Weinheim   
45014   2ND FLOOR REAR    07/05/2017             Justin          Grossman   
46252              NaN    02/15/2017             Ileana            Rivera   
58362               11    10/09/2019            Darlene            Barone   
64929    1ST FLOOR APT    11/28/2018            Darlene            Barone   

      RESIDENTIAL_COMMERCIAL_IND   BOROUGH  EVICTION_ZIP  \
32411                Residential     BRONX         10457   
45014                Residential  BROOKLYN         11207   
46252                Residential  BROOKLYN         11226   
58362                Residential     BRONX         10464   
64929                Residential     BRONX         10460   

                                          parsed_address  \
32411  OrderedDict([('AddressNumber', '2105'), ('Stre...   
45014  OrderedDict([('AddressNumber', '558'), ('Stree...   
46252  OrderedDict([('AddressNumber', '368'), ('Stree...   
58362  OrderedDict([('AddressNumber', '186'), ('Stree...   
64929  OrderedDict([('AddressNumber', '1495'), ('Stre...   

             address_cleaned eviction_address2  \
32411       2105 RYER AVENUE               NaN   
45014    558 FOUNTAIN AVENUE               NaN   
46252   368 EAST 26TH STREET               NaN   
58362       186 CROSS STREET               NaN   
64929     1495 BRYANT AVENUE               NaN   

                                     full_address lat_lon  \
32411          2105 RYER AVENUE, BRONX, NY, 10457     NaN   
45014    558 FOUNTAIN AVENUE, BROOKLYN, NY, 11207     NaN   
46252   368 EAST 26TH STREET, BROOKLYN, NY, 11226     NaN   
58362          186 CROSS STREET, BRONX, NY, 10464     NaN   
64929        1495 BRYANT AVENUE, BRONX, NY, 10460     NaN   

                                       lat_lon2  
32411   (40.853705649999995, -73.9008542867503)  
45014          (40.6662005, -73.87171695952176)  
46252          (40.6413199, -73.95270614900264)  
58362  (40.853431549999996, -73.78828781727552)  
64929           (40.832719, -73.88759423891094)

In [156]:
#reappend sub data to main data
data=data.join(sub_data,rsuffix='_xyx_')

In [157]:
extra_columns=data.columns[data.columns.str.contains('_xyx_')]
data=data.drop(columns=extra_columns)

In [158]:
data=data.drop(columns=extra_columns)

In [159]:
data[data['lat_lon'].isna()]

COURT_INDEX_NUMBER  DOCKET_NUMBER      EVICTION_ADDRESS  \
32411           59490/16         462842      2105 RYER AVENUE   
45014           63770/17          63865   558 FOUNTAIN AVENUE   
46252          K64852/16          72210  368 EAST 26TH STREET   
58362          B23662/19         116727      186 CROSS STREET   
64929          B35564/18         109605    1495 BRYANT AVENUE   

      EVICTION_APT_NUM EXECUTED_DATE MARSHAL_FIRST_NAME MARSHAL_LAST_NAME  \
32411               2A    01/09/2017              Danny          Weinheim   
45014   2ND FLOOR REAR    07/05/2017             Justin          Grossman   
46252              NaN    02/15/2017             Ileana            Rivera   
58362               11    10/09/2019            Darlene            Barone   
64929    1ST FLOOR APT    11/28/2018            Darlene            Barone   

      RESIDENTIAL_COMMERCIAL_IND   BOROUGH  EVICTION_ZIP  \
32411                Residential     BRONX         10457   
45014                Residential  BROOKLYN         11207   
46252                Residential  BROOKLYN         11226   
58362                Residential     BRONX         10464   
64929                Residential     BRONX         10460   

                                          parsed_address  \
32411  OrderedDict([('AddressNumber', '2105'), ('Stre...   
45014  OrderedDict([('AddressNumber', '558'), ('Stree...   
46252  OrderedDict([('AddressNumber', '368'), ('Stree...   
58362  OrderedDict([('AddressNumber', '186'), ('Stree...   
64929  OrderedDict([('AddressNumber', '1495'), ('Stre...   

             address_cleaned eviction_address2  \
32411       2105 RYER AVENUE               NaN   
45014    558 FOUNTAIN AVENUE               NaN   
46252   368 EAST 26TH STREET               NaN   
58362       186 CROSS STREET               NaN   
64929     1495 BRYANT AVENUE               NaN   

                                     full_address lat_lon  \
32411          2105 RYER AVENUE, BRONX, NY, 10457     NaN   
45014    558 FOUNTAIN AVENUE, BROOKLYN, NY, 11207     NaN   
46252   368 EAST 26TH STREET, BROOKLYN, NY, 11226     NaN   
58362          186 CROSS STREET, BRONX, NY, 10464     NaN   
64929        1495 BRYANT AVENUE, BRONX, NY, 10460     NaN   

                                       lat_lon2  
32411   (40.853705649999995, -73.9008542867503)  
45014          (40.6662005, -73.87171695952176)  
46252          (40.6413199, -73.95270614900264)  
58362  (40.853431549999996, -73.78828781727552)  
64929           (40.832719, -73.88759423891094)

In [160]:
data['lat_lon_both']=np.where(data['lat_lon'].isna(),data['lat_lon2'],data['lat_lon'])

In [161]:
data['lat_lon_both'].isna().mean()

0.0

In [163]:
#perfect return on coordinates isn't possible because of uncleaned addresses
data.isna().mean()

COURT_INDEX_NUMBER            0.000000
DOCKET_NUMBER                 0.000000
EVICTION_ADDRESS              0.000000
EVICTION_APT_NUM              0.165353
EXECUTED_DATE                 0.000000
MARSHAL_FIRST_NAME            0.000000
MARSHAL_LAST_NAME             0.000000
RESIDENTIAL_COMMERCIAL_IND    0.000000
BOROUGH                       0.000000
EVICTION_ZIP                  0.000000
parsed_address                0.000166
address_cleaned               0.006647
eviction_address2             0.999970
full_address                  0.000166
lat_lon                       0.000075
lat_lon2                      0.999925
lat_lon_both                  0.000000
dtype: float64

In [164]:
#a few errors from missing adddresses
data[data['full_address'].isna()]

COURT_INDEX_NUMBER  DOCKET_NUMBER  \
6625          B900783/17         381787   
20459          901264/17          72694   
21664           54821/16            803   
28001           67734/18          33804   
41996           72385/18         485910   
44470          K63535/17          79937   
47416         13182/15-1         204117   
52188          901800/17          76224   
57672         13182/15-2         198169   
58531           57909/19          23523   
59693           57521/19          17473   

                                        EVICTION_ADDRESS  \
6625                 333 EAST 181ST ST 355 EAST 184TH ST   
20459                    1972&1974&1976 PROSP ECT AVENUE   
21664  218-26,28A,28B MERRI CK BLVD  IN THE BLDG KNOW...   
28001              200-04 46TH ROAD ENT. 1 FAM. DWELLING   
41996           4927 BROADWAY IN THE  BLDG KA 616 WEST *   
44470                   1000 JAMAICA AVE 998 JAMAICA AVE   
47416  126-74,126-76,126-82 WILLETS POINT BLVD BUILDI...   
52188  531 COSTER ST IN  TH E BLDG. KNOWN AS 531-537 ...   
57672  126-74 126-76 126-82 WILLETS POINT BLVD BUILDI...   
58531                  91-01 /91-03/ 91-05 ASTORIA BLVD.   
59693                   912 SEVENTH AVE 200 WEST 58TH ST   

           EVICTION_APT_NUM EXECUTED_DATE MARSHAL_FIRST_NAME  \
6625                   COMM    08/22/2018            Richard   
20459  UNIT:%SUPERMARKET/GR    11/15/2018             Justin   
21664                   NaN    07/30/2018         Salavatore   
28001                   NaN    12/04/2018              Bruce   
41996                     *    02/01/2019              Danny   
44470            STOREFRONT    08/29/2017             Ileana   
47416                   NaN    09/17/2018            Richard   
52188             1ST FLOOR    09/11/2018             Justin   
57672                   NaN    09/17/2018            Richard   
58531                   NaN    06/05/2019             Edward   
59693                   NaN    10/07/2019             George   

      MARSHAL_LAST_NAME RESIDENTIAL_COMMERCIAL_IND    BOROUGH  EVICTION_ZIP  \
6625              McCoy                Residential      BRONX         10458   
20459          Grossman                 Commercial      BRONX         10460   
21664            Giglio                 Commercial     QUEENS         11413   
28001              Kemp                Residential     QUEENS         11361   
41996          Weinheim                 Commercial  MANHATTAN         10034   
44470            Rivera                 Commercial   BROOKLYN         11208   
47416           Capuano                 Commercial     QUEENS         11368   
52188          Grossman                 Commercial      BRONX         10474   
57672           Capuano                 Commercial     QUEENS         11368   
58531             Guida                 Commercial     QUEENS         11369   
59693       Essock, Jr.                 Commercial  MANHATTAN         10019   

      parsed_address address_cleaned eviction_address2 full_address  \
6625             NaN             NaN               NaN          NaN   
20459            NaN             NaN               NaN          NaN   
21664            NaN             NaN               NaN          NaN   
28001            NaN             NaN               NaN          NaN   
41996            NaN             NaN               NaN          NaN   
44470            NaN             NaN               NaN          NaN   
47416            NaN             NaN               NaN          NaN   
52188            NaN             NaN               NaN          NaN   
57672            NaN             NaN               NaN          NaN   
58531            NaN             NaN               NaN          NaN   
59693            NaN             NaN               NaN          NaN   

                                        lat_lon lat_lon2  \
6625   (18.776095734718467, 100.77393496701143)      NaN   
20459  (18.776095734718467, 100.77393496701143)      NaN   
21664  (18.776095

In [166]:
data['lat_lon_both']=data['lat_lon_both'].replace('(18.776095734718467, 100.77393496701143)',np.nan)

In [167]:
data['lat']=data['lat_lon_both'].str.replace('\(','').str.replace('\)','').str.split(', ',expand=True)[0].astype(float)
data['lon']=data['lat_lon_both'].str.replace('\(','').str.replace('\)','').str.split(', ',expand=True)[1].astype(float)

In [168]:
#Aside from the missing adddresses, the rest of the data should be approximately NYC lattitude
data[(data['lat']>41) | (data['lat']<40)]

Empty DataFrame
Columns: [COURT_INDEX_NUMBER, DOCKET_NUMBER, EVICTION_ADDRESS, EVICTION_APT_NUM, EXECUTED_DATE, MARSHAL_FIRST_NAME, MARSHAL_LAST_NAME, RESIDENTIAL_COMMERCIAL_IND, BOROUGH, EVICTION_ZIP, parsed_address, address_cleaned, eviction_address2, full_address, lat_lon, lat_lon2, lat_lon_both, lat, lon]
Index: []

In [169]:
data[(data['lon']>-72) | (data['lon']<-75)]

Empty DataFrame
Columns: [COURT_INDEX_NUMBER, DOCKET_NUMBER, EVICTION_ADDRESS, EVICTION_APT_NUM, EXECUTED_DATE, MARSHAL_FIRST_NAME, MARSHAL_LAST_NAME, RESIDENTIAL_COMMERCIAL_IND, BOROUGH, EVICTION_ZIP, parsed_address, address_cleaned, eviction_address2, full_address, lat_lon, lat_lon2, lat_lon_both, lat, lon]
Index: []

In [6]:
file_location = r'C:\Users\Adam Parente\Documents\Python Projects\new evictions\evictions_geocoded_2.csv'
data = pd.read_csv(file_location,index_col='Unnamed: 0')

C:\Users\Public\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
data['EXECUTED_DATE'].head()

0    09/12/2017
1    06/03/2019
2    11/16/2017
3    02/05/2019
4    06/07/2019
Name: EXECUTED_DATE, dtype: object

In [8]:
data['EXECUTED_DATE']=data['EXECUTED_DATE'].str.replace('2070','2020')

In [9]:
data['EXECUTED_DATE']=pd.to_datetime(data['EXECUTED_DATE'],utc=False)

In [10]:
data['EXECUTED_DATE'].dt.year

0        2017
1        2019
2        2017
3        2019
4        2019
         ... 
66344    2017
66345    2020
66346    2019
66347    2018
66348    2018
Name: EXECUTED_DATE, Length: 66349, dtype: int64

In [11]:
data['Executed_Year']=data['EXECUTED_DATE'].dt.year

In [12]:
data['Executed_Month']=data['EXECUTED_DATE'].dt.month

In [15]:
data['executed_year_month']=pd.to_datetime(data['Executed_Month'].astype(str)+"/"+data['Executed_Year'].astype(str))

No glaring outliers and the vast majority of addresses are used.

In [16]:
data.head()

COURT_INDEX_NUMBER  DOCKET_NUMBER             EVICTION_ADDRESS  \
0           50180/17           5469  463 LEXINGTON AVENUE APT. 1   
1          73750/18B          93847          448 SAPPHIRE STREET   
2          K65227/17          97709            1036 PRESIDENT ST   
3           51881/18         350940         2124 ST. PAUL AVENUE   
4           66981/18         351548            1098 GRANT AVENUE   

  EVICTION_APT_NUM EXECUTED_DATE MARSHAL_FIRST_NAME MARSHAL_LAST_NAME  \
0                1    2017-09-12             Robert          Renzulli   
1                1    2019-06-03             Justin          Grossman   
2              B2B    2017-11-16            Darlene            Barone   
3        2ND FLOOR    2019-02-05             Thomas               Bia   
4                2    2019-06-07             Thomas               Bia   

  RESIDENTIAL_COMMERCIAL_IND   BOROUGH  EVICTION_ZIP  ... eviction_address2  \
0                Residential  BROOKLYN         11221  ...               NaN   
1                Residential  BROOKLYN         11208  ...               NaN   
2                Residential  BROOKLYN         11225  ...               NaN   
3                Residential     BRONX         10461  ...               NaN   
4                Residential     BRONX         10456  ...               NaN   

                                 full_address  \
0   463 LEXINGTON AVENUE, BROOKLYN, NY, 11221   
1    448 SAPPHIRE STREET, BROOKLYN, NY, 11208   
2      1036 PRESIDENT ST, BROOKLYN, NY, 11225   
3      2124 ST. PAUL AVENUE, BRONX, NY, 10461   
4         1098 GRANT AVENUE, BRONX, NY, 10456   

                                    lat_lon lat_lon2  \
0   (40.68865999611498, -73.94358600207771)      NaN   
1    (40.66745998368947, -73.8567509939227)      NaN   
2  (40.668459986647804, -73.95718496942762)      NaN   
3   (40.85483998565695, -73.82997902754228)      NaN   
4   (40.83090001290239, -73.91641002796423)      NaN   

                               lat_lon_both       lat        lon  \
0   (40.68865999611498, -73.94358600207771)  40.68866 -73.943586   
1    (40.66745998368947, -73.8567509939227)  40.66746 -73.856751   
2  (40.668459986647804, -73.95718496942762)  40.66846 -73.957185   
3   (40.85483998565695, -73.82997902754228)  40.85484 -73.829979   
4   (40.83090001290239, -73.91641002796423)  40.83090 -73.916410   

   Executed_Year  Executed_Month  executed_year_month  
0           2017               9           2017-09-01  
1           2019               6           2019-06-01  
2           2017              11           2017-11-01  
3           2019               2           2019-02-01  
4           2019               6           2019-06-01  

[5 rows x 22 columns]

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66349 entries, 0 to 66348
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   COURT_INDEX_NUMBER          66349 non-null  object        
 1   DOCKET_NUMBER               66349 non-null  int64         
 2   EVICTION_ADDRESS            66349 non-null  object        
 3   EVICTION_APT_NUM            55378 non-null  object        
 4   EXECUTED_DATE               66349 non-null  datetime64[ns]
 5   MARSHAL_FIRST_NAME          66349 non-null  object        
 6   MARSHAL_LAST_NAME           66349 non-null  object        
 7   RESIDENTIAL_COMMERCIAL_IND  66349 non-null  object        
 8   BOROUGH                     66349 non-null  object        
 9   EVICTION_ZIP                66349 non-null  int64         
 10  parsed_address              66338 non-null  object        
 11  address_cleaned             65908 non-null  object    

In [18]:
data.to_csv(r"evictions_geocoded_2.csv")